# Several Datasets as Code Generation Evaluation Candidates

In [1]:
import datasets
import json
import ast
%load_ext autoreload
%autoreload 2

## LBPP

In [2]:
lbpp = datasets.load_dataset("CohereForAI/lbpp")

In [3]:
lbpp['test']

Dataset({
    features: ['task_id', 'language', 'title', 'instruction', 'completion', 'test_setup', 'test_list', 'signature', 'categories'],
    num_rows: 162
})

In [4]:
counter = 0
max_num = 50
for ddict in lbpp['test']:
    if counter == max_num:
        break
    for k2, v2 in ddict.items():
        print(k2)
        if k2 == 'test_list':
            alist = ast.literal_eval(v2)
            print('\n'.join(alist))
        else:
            print(v2)
        print('\n', '-'*75, '\n', sep='')
    counter += 1
    print('\n', '='*100, '\n', sep='')

task_id
lbpp/0

---------------------------------------------------------------------------

language
python

---------------------------------------------------------------------------

title
add_avg_and_std_cols_numpy

---------------------------------------------------------------------------

instruction
Write a python function `add_avg_and_std_cols_numpy(ar: np.ndarray) -> np.ndarray` that takes a 2D numpy array and returns a 2D numpy array with two additional columns appended to the end. The first column should contain the average of each row, and the second column should contain the standard deviation of each row. You may assume that the input array has at least one row and one column.

---------------------------------------------------------------------------

completion
import numpy as np


def add_avg_and_std_cols_numpy(ar: np.ndarray) -> np.ndarray:
    avg = np.mean(ar, axis=1).reshape(-1, 1)
    std = np.std(ar, axis=1).reshape(-1, 1)
    return np.hstack((ar, avg, std))


In [5]:
no_func = [i['instruction'] for i in lbpp['test'] if 'def ' not in i['instruction'] and 'write a python function' not in i['instruction'].lower()]
print(len(no_func))
print('\n\n'.join(no_func))

79
Given a list of unique words each of size k and an n sized word, w, where n is a multiple of k,
write a program in python to determine the number of unique combinations of words in the list that can be concatenated to form an anagram of the word w.

You are given a target word and an array of candidate words. Write a Python program to pick a subset of words from the candidate words such that 1) You can form the target word from the subset of candidate words by re-arranging some or all the letters of the candidate words. 2) The number of unused letters from the subset of candidate words is minimized. It is given that the number of candidate words is less than 20. Return the minimum number of unused letters for such a subset of candidate words.

You are given a 2d array of integers consisting of the heights of students in each grade. The first dimension of the array represents the grades and the second dimension represents the heights of students in that grade. Write a Python program 

In [ ]:
# better mention unit tests as in MBPP or provide the function name as in HumanEval?

## Big Code Bench

Something I could use. Need to figure out how to use the test cases correctly!

Much more manageable size - 20MB on dist vs. 10GB by Live Coding Bench

In [147]:
big = datasets.load_dataset("bigcode/bigcodebench")

Generating v0.1.0_hf split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Generating v0.1.1 split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Generating v0.1.2 split:   0%|          | 0/1140 [00:00<?, ? examples/s]

In [148]:
# saved to disk
from datasets import load_from_disk
big.save_to_disk('datasets/big-code-bench')

Saving the dataset (0/1 shards):   0%|          | 0/1140 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1140 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1140 [00:00<?, ? examples/s]

In [149]:
big

DatasetDict({
    v0.1.0_hf: Dataset({
        features: ['task_id', 'complete_prompt', 'instruct_prompt', 'canonical_solution', 'code_prompt', 'test', 'entry_point', 'doc_struct', 'libs'],
        num_rows: 1140
    })
    v0.1.1: Dataset({
        features: ['task_id', 'complete_prompt', 'instruct_prompt', 'canonical_solution', 'code_prompt', 'test', 'entry_point', 'doc_struct', 'libs'],
        num_rows: 1140
    })
    v0.1.2: Dataset({
        features: ['task_id', 'complete_prompt', 'instruct_prompt', 'canonical_solution', 'code_prompt', 'test', 'entry_point', 'doc_struct', 'libs'],
        num_rows: 1140
    })
})

In [154]:
min_value, max_value = 0, 25
for i, sample in enumerate(big['v0.1.2']):
    if i < min_value:
        continue
    if i > max_value:
        break
    print(f'Sample {i}')
    for k,v in sample.items():        
        print(f'{k}:\n{v}')
        print('\n', '-'*50, '\n', sep='')
    print('\n', '='*100, '\n', sep='')

Sample 0
task_id:
BigCodeBench/0

--------------------------------------------------

complete_prompt:
import itertools
from random import shuffle

def task_func(numbers=list(range(1, 3))):
    """
    Calculates the average of the sums of absolute differences between each pair of consecutive numbers 
    for all permutations of a given list. Each permutation is shuffled before calculating the differences.

    Args:
    - numbers (list): A list of numbers. Default is numbers from 1 to 10.
    
    Returns:
    float: The average of the sums of absolute differences for each shuffled permutation of the list.

    Requirements:
    - itertools
    - random.shuffle

    Example:
    >>> result = task_func([1, 2, 3])
    >>> isinstance(result, float)
    True
    """


--------------------------------------------------

instruct_prompt:
Calculates the average of the sums of absolute differences between each pair of consecutive numbers for all permutations of a given list. Each permutation 

### See prompt building scenarios and the use of unit tests in the GPT converstaions

## Live Code Bench
Holistic and Contamination Free Evaluation of Large Language Models for Code. LiveCodeBench collects new problems over time from contests across three competition platforms, namely LeetCode, AtCoder, and CodeForces. Currently, LiveCodeBench hosts four hundred high-quality coding problems that were published between May 2023 and May 2024. 

1. Regular dataset - 9.38GB in size! Problem description are long and logically ambiguous. There are only 400 problems, and the reason the file size is so big is that __test cases are so long that some don't fit into the Jupyter Notebook cell causing output errors or requiring to restart the page__
2. Lite dataset - 2GB on disk. Medium-level problems are still pretty hard coding challenges with long descriptions. Private test cases are scrambled and look unusable.
3. Test generation dataset ("livecodebench/test_generation") - ~400 problems that have a description of reasonable length. Only ~300KB on disk. Cons - unknown origin (synthetic?), no solution offered, each problem has __only one test case__. Used for the task of predicting the test output without actually generating the code of the function.

Leaderboard and code: https://livecodebench.github.io/ (large LLMs prevail!)

In [90]:
# regular dataset - 9.38G in size!
live = datasets.load_dataset("livecodebench/code_generation")

Generating test split: 0 examples [00:00, ? examples/s]

In [92]:
# saved to disk
from datasets import load_from_disk
live.save_to_disk('datasets/live-coding-bench')

Saving the dataset (0/18 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

In [91]:
live

DatasetDict({
    test: Dataset({
        features: ['question_title', 'question_content', 'platform', 'question_id', 'contest_id', 'contest_date', 'starter_code', 'difficulty', 'public_test_cases', 'private_test_cases', 'metadata'],
        num_rows: 400
    })
})

In [97]:
live['test'].shape, live['test'].column_names

((400, 11),
 ['question_title',
  'question_content',
  'platform',
  'question_id',
  'contest_id',
  'contest_date',
  'starter_code',
  'difficulty',
  'public_test_cases',
  'private_test_cases',
  'metadata'])

In [99]:
# difficulty level
set([i['difficulty'] for i in live['test']])

{'easy', 'hard', 'medium'}

In [100]:
# platform
set([i['platform'] for i in live['test']])

{'atcoder', 'codeforces', 'leetcode'}

In [105]:
medium_tasks = [i for i in live['test'] if i['difficulty']=='medium']
print(len(medium_tasks))

168


In [123]:
cols = ['question_title', 'question_content', 'contest_date', 'starter_code', 'difficulty', 'public_test_cases',]
for i, sample in enumerate(medium_tasks):
    for col in cols:
        print(f'{col}:\n{sample[col]}')

question_title:
B. Chemistry
question_content:
You are given a string $s$ of length $n$, consisting of lowercase Latin letters, and an integer $k$.

You need to check if it is possible to remove exactly $k$ characters from the string $s$ in such a way that the remaining characters can be rearranged to form a palindrome. Note that you can reorder the remaining characters in any way.

A palindrome is a string that reads the same forwards and backwards. For example, the strings "z", "aaa", "aba", "abccba" are palindromes, while the strings "codeforces", "reality", "ab" are not.

Input

Each test consists of multiple test cases. The first line contains a single integer $t$ ($1 \leq t \leq 10^4$) — the number of the test cases. This is followed by their description.

The first line of each test case contains two integers $n$ and $k$ ($0 \leq k < n \leq 10^5$) — the length of the string $s$ and the number of characters to be deleted.

The second line of each test case contains a string $s$ o

In [114]:
# lite version
live_lite = datasets.load_dataset("livecodebench/code_generation_lite", version_tag="release_v2")

The repository for livecodebench/code_generation_lite contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/livecodebench/code_generation_lite.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating test split: 0 examples [00:00, ? examples/s]

In [115]:
# saved to disk
from datasets import load_from_disk
live_lite.save_to_disk('datasets/live-lite-coding-bench')

Saving the dataset (0/4 shards):   0%|          | 0/511 [00:00<?, ? examples/s]

In [116]:
live_lite

DatasetDict({
    test: Dataset({
        features: ['question_title', 'question_content', 'platform', 'question_id', 'contest_id', 'contest_date', 'starter_code', 'difficulty', 'public_test_cases', 'private_test_cases', 'metadata'],
        num_rows: 511
    })
})

The lite dataset has more rows than the regular one, but the saved file is much smaller - only 2GB.

In [117]:
live_lite['test'].shape, live_lite['test'].column_names

((511, 11),
 ['question_title',
  'question_content',
  'platform',
  'question_id',
  'contest_id',
  'contest_date',
  'starter_code',
  'difficulty',
  'public_test_cases',
  'private_test_cases',
  'metadata'])

In [99]:
# difficulty level
set([i['difficulty'] for i in live_lite['test']])

{'easy', 'hard', 'medium'}

In [118]:
# platform
set([i['platform'] for i in live_lite['test']])

{'atcoder', 'codeforces', 'leetcode'}

In [119]:
medium_tasks = [i for i in live_lite['test'] if i['difficulty']=='medium']
print(len(medium_tasks))

206


In [122]:
medium_tasks[2]

{'question_title': 'sum-in-a-matrix',
 'question_content': 'You are given a 0-indexed 2D integer array nums. Initially, your score is 0. Perform the following operations until the matrix becomes empty:\n\nFrom each row in the matrix, select the largest number and remove it. In the case of a tie, it does not matter which number is chosen.\nIdentify the highest number amongst all those removed in step 1. Add that number to your score.\n\nReturn the final score.\n \nExample 1:\n\nInput: nums = [[7,2,1],[6,4,2],[6,5,3],[3,2,1]]\nOutput: 15\nExplanation: In the first operation, we remove 7, 6, 6, and 3. We then add 7 to our score. Next, we remove 2, 4, 5, and 2. We add 5 to our score. Lastly, we remove 1, 2, 3, and 1. We add 3 to our score. Thus, our final score is 7 + 5 + 3 = 15.\n\nExample 2:\n\nInput: nums = [[1]]\nOutput: 1\nExplanation: We remove 1 and add it to the answer. We return 1.\n \nConstraints:\n\n1 <= nums.length <= 300\n1 <= nums[i].length <= 500\n0 <= nums[i][j] <= 10^3',
 

In [125]:
cols = ['question_title', 'question_content', 'contest_date', 'starter_code', 'difficulty', 'public_test_cases',]
min_value, max_value = 5, 10
for i, sample in enumerate(medium_tasks):
    if i < min_value:
        continue
    if i > max_value:
        break
    print(f'Sample {i}')
    for col in cols:
        print(f'{col}:\n{sample[col]}')

Sample 5
question_title:
extra-characters-in-a-string
question_content:
You are given a 0-indexed string s and a dictionary of words dictionary. You have to break s into one or more non-overlapping substrings such that each substring is present in dictionary. There may be some extra characters in s which are not present in any of the substrings.
Return the minimum number of extra characters left over if you break up s optimally.
 
Example 1:

Input: s = "leetscode", dictionary = ["leet","code","leetcode"]
Output: 1
Explanation: We can break s in two substrings: "leet" from index 0 to 3 and "code" from index 5 to 8. There is only 1 unused character (at index 4), so we return 1.


Example 2:

Input: s = "sayhelloworld", dictionary = ["hello","world"]
Output: 3
Explanation: We can break s in two substrings: "hello" from index 3 to 7 and "world" from index 8 to 12. The characters at indices 0, 1, 2 are not used in any substring and thus are considered as extra characters. Hence, we return 

In [127]:
test = datasets.load_dataset("livecodebench/test_generation")

Generating test split:   0%|          | 0/442 [00:00<?, ? examples/s]

In [140]:
# saved to disk
from datasets import load_from_disk
test.save_to_disk('datasets/live-coding-bench-test-generation')

Saving the dataset (0/1 shards):   0%|          | 0/442 [00:00<?, ? examples/s]

In [128]:
test

DatasetDict({
    test: Dataset({
        features: ['question_title', 'question_content', 'question_id', 'contest_id', 'test_id', 'contest_date', 'starter_code', 'function_name', 'difficulty', 'test'],
        num_rows: 442
    })
})

The lite dataset has more rows than the regular one, but the saved file is much smaller - only 2GB.

In [131]:
test['test'].shape, test['test'].column_names

((442, 10),
 ['question_title',
  'question_content',
  'question_id',
  'contest_id',
  'test_id',
  'contest_date',
  'starter_code',
  'function_name',
  'difficulty',
  'test'])

In [132]:
# difficulty level
set([i['difficulty'] for i in test['test']])

{'easy', 'hard', 'medium'}

In [133]:
medium_tasks = [i for i in test['test'] if i['difficulty']=='medium']
print(len(medium_tasks))

223


In [139]:
medium_tasks[200]

{'question_title': 'find-longest-special-substring-that-occurs-thrice-i',
 'question_content': 'You are given a string s that consists of lowercase English letters.\nA string is called special if it is made up of only a single character. For example, the string "abc" is not special, whereas the strings "ddd", "zz", and "f" are special.\nReturn the length of the longest special substring of s which occurs at least thrice, or -1 if no special substring occurs at least thrice.\nA substring is a contiguous non-empty sequence of characters within a string.\n \n',
 'question_id': '3267',
 'contest_id': 'weekly-contest-378',
 'test_id': 2,
 'contest_date': Timestamp('2023-12-30 00:00:00'),
 'starter_code': 'def maximumLength(self, s: str) -> int:\n    pass',
 'function_name': 'maximumLength',
 'difficulty': 'medium',
 'test': '[{"input": "\\"abcaba\\"", "output": "1", "testtype": "functional"}]'}

In [144]:
min_value, max_value = 10, 25
for i, sample in enumerate(medium_tasks):
    if i < min_value:
        continue
    if i > max_value:
        break
    print(f'Sample {i}')
    for k, v in sample.items():
        print(f'{k}:\n{v}')
        print('\n', '-'*50, '\n', sep='')
    print('\n', '='*100, '\n', sep='')

Sample 10
question_title:
find-the-longest-semi-repetitive-substring

--------------------------------------------------

question_content:
You are given a 0-indexed string s that consists of digits from 0 to 9.
A string t is called a semi-repetitive if there is at most one consecutive pair of the same digits inside t. For example, 0010, 002020, 0123, 2002, and 54944 are semi-repetitive while 00101022, and 1101234883 are not.
Return the length of the longest semi-repetitive substring inside s.
A substring is a contiguous non-empty sequence of characters within a string.
 


--------------------------------------------------

question_id:
2786

--------------------------------------------------

contest_id:
biweekly-contest-106

--------------------------------------------------

test_id:
0

--------------------------------------------------

contest_date:
2023-06-10 00:00:00

--------------------------------------------------

starter_code:
def longestSemiRepetitiveSubstring(self, s: s

## TACO

See `References_Praxis.doc` for more details about the dataset.

The dataset contains coding challenges from:
* HackerRank
* Leetcode
* GeeksForGeeks
* Codewars
* Hackerearth and others

Dataset can be filtered by:
* Difficulty - 'EASY', 'HARD', 'MEDIUM', 'MEDIUM_HARD', 'UNKNOWN_DIFFICULTY', 'VERY_HARD'
* dates - to get the freshest problems to overcome the data leakage issue;
* tags and raw tags (different types of problems); skills are part of tags;

__THE DATASET CONTAINS COLUTIONS__

```
# non-empty solutions and input_output features can be parsed from text format this way:
sample = next(iter(taco['train']))
sample["solutions"] = json.loads(sample["solutions"])
sample["input_output"] = json.loads(sample["input_output"])
sample["raw_tags"] = eval(sample["raw_tags"])
sample["tags"] = eval(sample["tags"])
sample["skill_types"] = eval(sample["skill_types"])
```

In [7]:
# load dataset from HuggingFace
taco = datasets.load_dataset("BAAI/TACO")

The repository for BAAI/TACO contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/BAAI/TACO.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/25443 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

__Saving a HuggingFace dataset:__  
[Link to howto notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/videos/save_load_dataset.ipynb)

The code below saves taco to the arrow format. See the above link for csv or json

In [17]:
from datasets import load_from_disk
taco.save_to_disk('datasets/taco-arrow')

# reload back to memory
#taco = load_from_disk('datasets/taco-arrow')

Saving the dataset (0/9 shards):   0%|          | 0/25443 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [29]:
# dataset structure
type(taco), taco, f'train: {len(taco["train"])}', f'test: {len(taco["test"])}'

(datasets.dataset_dict.DatasetDict,
 DatasetDict({
     train: Dataset({
         features: ['question', 'solutions', 'starter_code', 'input_output', 'difficulty', 'raw_tags', 'name', 'source', 'tags', 'skill_types', 'url', 'Expected Auxiliary Space', 'time_limit', 'date', 'picture_num', 'memory_limit', 'Expected Time Complexity'],
         num_rows: 25443
     })
     test: Dataset({
         features: ['question', 'solutions', 'starter_code', 'input_output', 'difficulty', 'raw_tags', 'name', 'source', 'tags', 'skill_types', 'url', 'Expected Auxiliary Space', 'time_limit', 'date', 'picture_num', 'memory_limit', 'Expected Time Complexity'],
         num_rows: 1000
     })
 }),
 'train: 25443',
 'test: 1000')

In [45]:
# look inside
print('Taining set shape:', taco["train"].shape)
print('\nColumn names:\n', '\n'.join(sorted(taco["train"].column_names)), sep='')

print('\nTest set shape:', taco["test"].shape)
print('\nColumn names:\n', '\n'.join(sorted(taco["test"].column_names)), sep='')

Taining set shape: (25443, 17)

Column names:
Expected Auxiliary Space
Expected Time Complexity
date
difficulty
input_output
memory_limit
name
picture_num
question
raw_tags
skill_types
solutions
source
starter_code
tags
time_limit
url

Test set shape: (1000, 17)

Column names:
Expected Auxiliary Space
Expected Time Complexity
date
difficulty
input_output
memory_limit
name
picture_num
question
raw_tags
skill_types
solutions
source
starter_code
tags
time_limit
url


__Irrelevant columns__:  
Expected Auxiliary Space, Expected Time Complexity, memory_limit, time_limit, url, source, name (name of the problem), picture_num


In [84]:
# sources
set([i['source'] for i in taco['train']])

{'aizu',
 'atcoder',
 'codechef',
 'codeforces',
 'codewars',
 'geeksforgeeks',
 'hackerearth',
 'hackerrank',
 'kattis',
 'leetcode'}

In [64]:
# difficulty level
set([i['difficulty'] for i in taco['train']])

{'EASY', 'HARD', 'MEDIUM', 'MEDIUM_HARD', 'UNKNOWN_DIFFICULTY', 'VERY_HARD'}

I could use the latest dates if I need the freshest problems (not used for training of LLMs) - a total of 1208 problems fresher than Jan 1, 2022:

In [71]:
# explore dates
dates = sorted(list(set([i['date'] for i in taco['train'] if i['date'] is not None])))
print('Unique dates:', len(dates))
print('First dates:', dates[:100])
print('Last dates:', dates[-100:])

# how many fresh problems
cutoff = '2022-01-01'
fresh = [i['date'] for i in taco['train'] if i['date'] is not None and i['date']>=cutoff]
print(f'Total number of problems fresher than {cutoff}:', len(fresh))

Unique dates: 1440
First dates: ['2008-12-01', '2009-03-15', '2009-04-20', '2009-05-28', '2009-06-05', '2009-06-18', '2009-07-28', '2009-07-30', '2009-09-15', '2009-10-15', '2010-04-10', '2010-05-03', '2010-05-05', '2010-08-06', '2010-08-09', '2010-10-28', '2010-11-07', '2010-11-09', '2010-11-10', '2010-11-12', '2010-11-23', '2010-12-03', '2010-12-04', '2011-01-13', '2011-01-14', '2011-02-02', '2011-02-21', '2011-03-06', '2011-03-07', '2011-04-07', '2011-04-12', '2011-04-13', '2011-04-21', '2011-05-02', '2011-05-10', '2011-05-18', '2011-06-14', '2011-07-13', '2011-07-14', '2011-07-29', '2011-08-02', '2011-08-22', '2011-08-27', '2011-09-02', '2011-09-06', '2011-09-07', '2011-10-17', '2011-11-04', '2011-11-11', '2011-12-05', '2011-12-26', '2012-01-09', '2012-01-17', '2012-01-18', '2012-01-20', '2012-01-24', '2012-02-17', '2012-02-18', '2012-02-26', '2012-02-27', '2012-03-01', '2012-03-03', '2012-03-11', '2012-03-20', '2012-03-21', '2012-03-25', '2012-03-28', '2012-04-03', '2012-04-06', '

In [79]:
# skills - part of tags
skills = [eval(i['skill_types']) for i in taco['train']]
skills = sorted(set([item for sublist in skills for item in sublist]))
skills

['Amortized analysis',
 'Bit manipulation',
 'Complete search',
 'Data structures',
 'Dynamic programming',
 'Greedy algorithms',
 'Range queries',
 'Sorting']

In [80]:
# tags
tags = [eval(i['tags']) for i in taco['train']]
tags = sorted(set([item for sublist in tags for item in sublist]))
tags

['Ad-hoc',
 'Amortized analysis',
 'Bit manipulation',
 'Combinatorics',
 'Complete search',
 'Constructive algorithms',
 'Data structures',
 'Directed graphs',
 'Divide and conquer',
 'Dynamic programming',
 'Flows and cuts',
 'Fundamentals',
 'Game theory',
 'Geometry',
 'Graph algorithms',
 'Graph traversal',
 'Greedy algorithms',
 'Implementation',
 'Mathematics',
 'Matrices',
 'Number theory',
 'Paths and circuits',
 'Polynomials and generating functions',
 'Preprocessing',
 'Probability',
 'Range queries',
 'Segment trees revisited',
 'Shortest paths',
 'Sorting',
 'Spanning trees',
 'Square root algorithms',
 'String algorithms',
 'Strong connectivity',
 'Sweep line algorithms',
 'Tree algorithms',
 'Tree queries']

In [81]:
# raw_tags - some are less meaningful
raw_tags = [eval(i['raw_tags']) for i in taco['train']]
raw_tags = sorted(set([item for sublist in raw_tags for item in sublist]))
raw_tags

['*special',
 '0-1 Knapsack',
 '1D Arrays',
 '2 SAT',
 '2-sat',
 '2D Arrays',
 '2d array',
 '3D+ Arrays',
 'AP',
 'ASCII Art',
 'AVL-Tree',
 'Activity Selection',
 'Ad-Hoc',
 'Advanced Algorithms',
 'Advanced DP',
 'Advanced Data Structure',
 'Advanced Data Structures',
 'Advanced Graph Algos',
 'Advanced Tree Structures',
 'Aho Corasick',
 'Algebra',
 'Algorithms',
 'Algorithms - Bit Manipulation',
 'Algorithms - Constructive Algorithms',
 'Algorithms - Debugging',
 'Algorithms - Dynamic Programming',
 'Algorithms - Game Theory',
 'Algorithms - Graph Theory',
 'Algorithms - Greedy',
 'Algorithms - Implementation',
 'Algorithms - NP Complete',
 'Algorithms - Recursion',
 'Algorithms - Search',
 'Algorithms - Sorting',
 'Algorithms - Strings',
 'Algorithms - Warmup',
 'Arbitrary Precision',
 'Arithmetic',
 'Array',
 'Arrays',
 'Auxiliary Tree',
 'BFS',
 'BIT',
 'Backtracking',
 'Basic Maths',
 'Basic Programming Concepts',
 'Basic-Programming',
 'BasicProgramming',
 'Beatty sequence',
 

In [83]:
# iterate train
medium_tasks = [i for i in taco['train'] if i['difficulty']=='MEDIUM']
print(len(medium_tasks))
min_value, max_value = 0, 25
for i, sample in enumerate(medium_tasks):
    if i < min_value:
        continue
    if i > max_value:
        break
    print(f'SAMPLE {i}\n')
    sample["input_output"] = json.loads(sample["input_output"])
    for k,v in sample.items():
        # print each input and each output next to each other
        if k=='input_output':
            print(f'{k}:')
            for k2, v2 in v.items():
                merged = "\n".join(v2)
                print(f'\n{k2}:\n{merged}')
        else:
            print(f'{k}:\n\n{v}')
            print('\n', '-'*50, '\n', sep='')
    print('\n', '='*100, sep='')
    print('='*100, sep='')
    print('='*100, '\n', sep='')

3244
SAMPLE 0

question:

Given a string 's'. The task is to find the smallest window length that contains all the characters of the given string at least one time.
For eg. A = aabcbcdbca, then the result would be 4 as of the smallest window will be dbca.
 
Example 1:
Input : "AABBBCBBAC"
Output : 3
Explanation : Sub-string -> "BAC"
Example 2:
Input : "aaab"
Output : 2
Explanation : Sub-string -> "ab"
 
Example 3:
Input : "GEEKSGEEKSFOR"
Output : 8
Explanation : Sub-string -> "GEEKSFOR"
 
Your Task:  
You don't need to read input or print anything. Your task is to complete the function findSubString() which takes the string  S as input and returns the length of the smallest such window of the string.
Expected Time Complexity: O(256.N)
Expected Auxiliary Space: O(256)
 
Constraints:
1 ≤ |S| ≤ 10^{5}
String may contain both type of English Alphabets.

--------------------------------------------------

solutions:

["class Solution:\n\n\tdef findSubString(self, str):\n\t\tdict = {}\n\t\ta

TypeError: sequence item 0: expected str instance, list found

In [87]:
# iterate test
medium_tasks = [i for i in taco['test'] if i['difficulty']=='MEDIUM']
print(len(medium_tasks))
min_value, max_value = 0, 5
for i, sample in enumerate(medium_tasks):
    if i < min_value:
        continue
    if i > max_value:
        break
    print(f'SAMPLE {i}\n')
    sample["input_output"] = json.loads(sample["input_output"])
    for k,v in sample.items():
        # print each input and each output next to each other
        if k=='input_output':
            print(f'{k}:')
            for k2, v2 in v.items():
                merged = "\n".join(v2)
                print(f'\n{k2}:\n{merged}')
        else:
            print(f'{k}:\n\n{v}')
            print('\n', '-'*50, '\n', sep='')
    print('\n', '='*100, sep='')
    print('='*100, sep='')
    print('='*100, '\n', sep='')

200
SAMPLE 0

question:

The city park of IT City contains n east to west paths and n north to south paths. Each east to west path crosses each north to south path, so there are n^2 intersections.

The city funded purchase of five benches. To make it seems that there are many benches it was decided to place them on as many paths as possible. Obviously this requirement is satisfied by the following scheme: each bench is placed on a cross of paths and each path contains not more than one bench.

Help the park administration count the number of ways to place the benches.


-----Input-----

The only line of the input contains one integer n (5 ≤ n ≤ 100) — the number of east to west paths and north to south paths.


-----Output-----

Output one integer — the number of ways to place the benches.


-----Examples-----
Input
5

Output
120

--------------------------------------------------

solutions:

["n = int(input())\ncn5 = n * (n - 1) // 2 * (n - 2) // 3 * (n - 3) // 4 * (n - 4) // 5\nan5 